In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
from data_agent import *
import numpy as np
from data_util import *
import tensorflow as tf
from sklearn_model_hyper import *
import pandas as pd
from stock_agent import *
from backtest import *
from joblib import dump, load
from datetime import datetime, timedelta
from tec_an import *
from bitstamp import *
from process_live_trade import *
from model_winner_select import *
from eval_model import *
from cache_providers import *

base_dir = "data/"

In [2]:
currency = "btcusd"

full_data = base_dir + "backtest/"
full_data = base_dir + "4jul21/"
#full_data = base_dir + "btcusd/"
data_gen = DataGenerator(random = False, base_dir = full_data)
data_gen.rewind()
data_count = (data_gen.steps - 100)
#data_count = 200000
raw = data_gen.next()

minute = 15

hot_load = True

In [ ]:
path = "model/"
files = os.listdir(path)
models = []
for file in files:
    models.append(f"{path}{file}")

models = ['model/rand']
#models = ['model/quadra']
models = ['model/mpl']
#models = ['model/cat']
    
val_start = 1629507199#int(raw['timestamp'])

In [ ]:
time_frame = (60*5)
results = load_bitstamp_ohlc_by_period(currency, 
                                    start=1627836747,
                                    end=1629737547,
                                    step=time_frame,
                                    verbose = True
                                    )

In [ ]:
last = results[0]

for r in results:
    diff = (int(r['timestamp']) - int(last['timestamp']))
    if (diff != time_frame):
        print(f"{diff} - {r['timestamp']}")
    last = r

In [ ]:
cache = CacheProvider([currency],  verbose = True)

online = cache.get_provider(15, 40, val_start = 1629507700, val_end = 1629538206)


In [ ]:

back, metric = eval_model(
    model = model, 
    currency = currency, 
    step = 1,
    verbose = True, 
    provider = online,
    hot_load_total = 100,
    cache = cache
    )

back.report()

In [ ]:
winner = get_best_model(
    minutes_list=[minute],
    result_paths=models,
    currency_list=[currency],
    timestamp = val_start,
    winner_path = None
)

In [ ]:
result = winner

model = result['model']
windows = result['window']
minutes = result['minutes']
step = result['step']
profit = result['profit']
print(f"Minutes={minutes} Window={windows} Step={step} | {profit}")
print(f"{model}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def get_charts(stock_data):
    fig = plt.figure()
    ax = plt.axes()
    x = stock_data
    plt.plot(x, linewidth=1)
    #plt.grid(b=True, color='aqua', alpha=0.3, linestyle='-.', linewidth=2)
    plt.show()


In [ ]:

on_raw_data = lambda raw: print(raw)

data_gen.rewind()

first = data_gen.next()

In [ ]:
agent_reference, back_reference, stock_reference = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True,
                                verbose = False)
prices = []
agent_reference.on_closed_price = lambda price: prices.append(price)
#agent.ohlc['open'].plot()

In [ ]:
data_gen.rewind()


reference_list = []
#data_count
for i in tqdm(range(data_count)):
    raw = data_gen.next()
    price = raw[PRICE_KEY]
    amount = raw[AMOUNT_KEY]
    timestamp = raw[TIMESTAMP_KEY]
    timestamp_pd = pd.to_datetime(timestamp, unit='s')
    reference_list.append([timestamp_pd, price])


In [ ]:
DATE = 'Date'
df = pd.DataFrame(agent_reference.list, columns = [DATE, CLOSE])
df = df.set_index(pd.DatetimeIndex(df[DATE]))

In [ ]:
time = df[CLOSE].resample(agent_reference.resample)
ohlc = time.ohlc()
print("All data reference")
#ohlc.plot()
#ohlc

In [ ]:
cache = CacheProvider([currency],  verbose = True)


In [ ]:
online = cache.get_provider(minutes, windows, val_start = 1629507700, val_end = 1629538206)

In [ ]:
cache = CacheProvider([currency],  verbose = True)

online = cache.get_provider(minutes, windows, val_start = 1629507700, val_end = 1629538206)

back, metric = eval_model(
    model = model, 
    currency = currency, 
    step = step,
    verbose = True, 
    provider = online,
    hot_load_total = 100,
    cache = cache
    )

back.report()

In [ ]:
agent_raw, back_raw, stock_raw = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                verbose = True,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True)
prices = []

In [ ]:
agent_raw.last_index

In [ ]:
agent_raw.save_history = True
stock_raw.verbose = True
raw = data_gen.next()
agent_raw.on_new_raw_data(raw)

In [ ]:
agent_raw.last_index

In [ ]:
data_gen.rewind()
agent_raw.save_history = True
stock_raw.verbose = True

init_raw_process(currency, minutes, int(first['timestamp']), agent_raw)

#data_count
for i in range(data_count):
    raw = data_gen.next()
    agent_raw.on_new_raw_data(raw)

In [ ]:
val_start = int(first['timestamp'])
val_end = int(raw['timestamp'])

start_string = pd.to_datetime(val_start, unit='s')
end_string = pd.to_datetime(val_end, unit='s')

print(f"{start_string}({val_start}) -> {end_string}({val_end})")

In [ ]:
back_raw.on_down(back_raw.buy_price, back_raw.buy_price)
back_raw.report()

In [ ]:
agent_raw.ohlc[-50:]["open"].plot()
#agent.ohlc[1:]

In [ ]:
x_list, price_list, time_list = online.load_val_data(currency)

In [ ]:
get_charts(price_list)
print(len(price_list))

In [ ]:
agent_array, back_array, stock_array = get_agent(minutes = minutes,
                                win = windows,
                                step = step,
                                currency = currency,
                                hot_load = hot_load,
                                model = model,
                                timestamp = int(first['timestamp']),
                                simulate_on_price = True,
                                verbose = False)

In [ ]:
agent_array.save_history = True
stock_array.verbose = True
for idx in range(len(price_list)):
    price = price_list[idx]
    time = time_list[idx]
    order = [[f"{price}", f"{price}"]]
    amount = 0.0
    agent_array.process_data(price, amount, time, order, order)

In [ ]:
back_array.on_down(back_array.buy_price, back_array.buy_price)
back_array.report()

In [ ]:
def eval_step_(model, train_set, step, provider, verbose = False):

    valX, valY, times = provider.load_val_data(train_set)
    
    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    history = []
    
    preds = model.predict(np.array(x))
    
    #for idx in range(len(x)):
    #    print(f"{times[idx]} -> {closed_prices[idx]} {y[idx]} ")

    metrics = {}
    metrics["recall"] = recall_score(y, preds)
    metrics["precision"] = precision_score(y, preds)
    metrics["f1"] = f1_score(y, preds)
    metrics["accuracy"] = accuracy_score(y, preds)
    metrics["roc_auc"] = roc_auc_score(y, preds)
    
    back = BackTest(value = 100, 
                    verbose = True, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    
    back = backtest_model(model, valX, valY, back)
    
    for idx in range(len(times)):
        timestamp = times[idx]
        price = closed_prices[idx]
        y = model.predict([valX[idx]])
        is_up = False
        if (y[0] > 0.5):
            is_up = True
        history.append((timestamp, price, x, is_up))
    
    get_charts(valY)

    
    return back, metrics, history
    

In [ ]:
back_eval, metr, history = eval_step_(model, currency, step, online, verbose = False)
back_eval.on_down(back_eval.buy_price, back_eval.buy_price)
back_eval.report()

In [ ]:
for data in history:
    print(f"{data[0]} - {data[1]} - {data[3]}")

In [ ]:
print(metrics)


In [ ]:
back_raw.report()
#-1.78465% -> 98.21535030676218

In [ ]:
back_array.report()
#-1.89717% -> 98.10282664807247

In [ ]:
agent_raw.report()

In [ ]:
agent_array.report()

In [3]:
trained_list = load("model/quad_3_5")

In [6]:
print(trained_list[-2])

TrainedModel (profit = 109.7516404930184, profit_per_currency = {'btcusd': 6.93582, 'ethusd': 25.17257, 'ltcusd': 2.75049, 'ltcbtc': 4.14768}, model_detail = ModelDetail (model = Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pipeline',
                 Pipeline(steps=[('s',
                                  RFE(estimator=ExtraTreesClassifier(max_features=0.3,
                                                                     random_state=42),
                                      step=0.7500000000000001)),
                                 ('m', QuadraticDiscriminantAnalysis())]))]), data_detail = DataDetail (windows = 5, minutes = 3, steps_ahead = 2)))


In [10]:
lala = {}
lala["d"] = 12

In [11]:
if "d" in lala:
    print("Exists")
else:
    print("Does not exist")

Exists
